<a href="https://colab.research.google.com/github/manaspatel-chy23iitbhu/Real_Time_Parking_Price/blob/main/Final__Manas_Patel_SA'25_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install pathway bokeh --quiet # This cell may take a few seconds to execute.
 #This cell is used to install pathway files

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathway as pw
import panel as pn
import datetime
from datetime import datetime
import bokeh
# We are importing all the necessary libraries which will be further used in the project

To not face any issue after chaging the name of lot if you wish to you should start running from these cells again

In [ ]:
#Dataset is read from csv file
url = "https://raw.githubusercontent.com/manaspatel-chy23iitbhu/Real_Time_Parking_Price/main/dataset.csv"
df = pd.read_csv(url)
print(df.shape) #Prints the number of rows and columns present in df
df.head() #Prints the first 5 rows of df

In [ ]:
#Merges Date and Time column into a single column and creates a new column with given format
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + " " + df['LastUpdatedTime'], format = "%d-%m-%Y %H:%M:%S")

In [ ]:
#Sorts the df by SystemCodeNumber first and then by the Timestamp column created in last step
df = df.sort_values(by = ['SystemCodeNumber','Timestamp']).reset_index(drop = True)

In [ ]:
#All the unique SystemCodeNumbers are stoed as an array and SystemCodeNumbers are printed to help user find the lot they are interested in
unique_lots = df["SystemCodeNumber"].unique()
unique_lots

In [ ]:
#A for loop is run, which creates a different csv according to each parking lot and names it accordingly
for lot in unique_lots:
    lot_df = df[df["SystemCodeNumber"] == lot].copy()
    lot_df.to_csv(f"parking_stream_{lot}.csv", index=False)

You can change the name of lot from here

In [ ]:
#df = pd.read_csv('parking_stream_"Put the name of lot system code here to create pricing model of that parking lot".csv')
#run all cells after cell no.4 after changing name
df = pd.read_csv('parking_stream_Shopping.csv')#here
df['TrafficConditionNearby'] = df['TrafficConditionNearby'].replace({'low':0,'high': 2,'average':1}).astype('int32')#replaces given data with numbers for calculation simplification
df['VehicleType'] = df['VehicleType'].replace({'car':2,'bike': 1,'truck':3,'cycle':0}).astype('int32')#replaces given data with numbers for calculation simplification
# After performing necessary preprocessing in the lot wise csv a final csv is created which will be used as a demo stream
df.to_csv('parking_data.csv', index = False)

In [ ]:
#A schema being created to support necessary data flow and define its type
class ParkingSchema(pw.Schema):
    ID: int
    SystemCodeNumber: str
    Capacity: int
    Latitude: float
    Longitude: float
    Occupancy: int
    VehicleType: int
    TrafficConditionNearby: int
    QueueLength: int
    IsSpecialDay: int
    LastUpdatedDate: str
    LastUpdatedTime: str
    Timestamp: str

In [ ]:
#A demo stream is created using pathway to play csv as data stream where input rate is 50 rows per second
data = pw.demo.replay_csv("parking_data.csv", schema=ParkingSchema, input_rate=50)

In [ ]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

Model 1

In [ ]:
# Define a daily tumbling window over the data stream using Pathway
# This block performs temporal aggregation and computes a dynamic price for each day
import datetime

delta_window = (
    data_with_time.windowby(
        pw.this.t,  # Event time column to use for windowing (parsed datetime)
        instance=pw.this.day,  # Logical partitioning key: one instance per calendar day
        window=pw.temporal.tumbling(datetime.timedelta(hours=0.5)),  # Fixed-size window of 0.5 hour
        behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
    )
    .reduce(
        t=pw.this._pw_window_end,                        # Assign the end timestamp of each window
        occ_avg=pw.reducers.avg(pw.this.Occupancy),     # Average occupancy observed in the window
        cap=pw.reducers.max(pw.this.Capacity),           # Maximum capacity observed (typically constant per lot)
    )
    .with_columns(
        # Compute the price using a simple dynamic pricing formula:
        #     price = base_price + demand_fluctuation
        #     where:
        #         base_price = 10 (fixed minimum price)
        #         demand_fluctuation = (occ_avg) / cap
        #
        # Intuition:
        #The closer the ratio of occupancy average and capacity to 1 the higher will be the demand, hence increasing the prices
        #The prices fluctuate between $10-$15 which is justifiable from demand and supply side of business and customers will too
        price=10 + 5*(pw.this.occ_avg) / (pw.this.cap)
    )
)


In [ ]:
from bokeh.plotting import figure #libraries necessary for plot are imported
import bokeh.plotting
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Daily Parking Price in $",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()

##Model **2**

In [ ]:
# Define a daily tumbling window over the data stream using Pathway
# This block performs temporal aggregation and computes a dynamic price for each day
delta_window1 = (
    data_with_time.windowby(
        pw.this.t,  # Event time column to use for windowing (parsed datetime)
        instance=pw.this.day,  # Logical partitioning key: one instance per calendar day
        window=pw.temporal.tumbling(datetime.timedelta(hours=0.5)),  # Fixed-size daily window of 0.5 hour
        behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
    )
    .reduce(
        t=pw.this._pw_window_end,                        # Assign the end timestamp of each window
        occ_avg=pw.reducers.avg(pw.this.Occupancy),     # Average occupancy observed in the window
        cap=pw.reducers.max(pw.this.Capacity),           # Maximum capacity observed (typically constant per spot)
        queue_length=pw.reducers.avg(pw.this.QueueLength),  # Average queue length in the window
        traffic=pw.reducers.any(pw.this.TrafficConditionNearby),  # Any traffic condition value in the window
        special_day=pw.reducers.any(pw.this.IsSpecialDay),  # Any special day flag in the window
        vehicle_type=pw.reducers.any(pw.this.VehicleType),  # Any vehicle type value in the window
    )
    .with_columns(
        # Demand Formula:
        #     demand = α * (Occupancy / Capacity) + β * QueueLength + γ * Traffic + δ * IsSpecialDay + ε * VehicleType
        #     where:
        #         α = 1.2, β = 0.2, γ = 0.2, δ = 0.8, ε = 0.2
        #         Occupancy, QueueLength, Traffic, etc., are normalized values
        # This keeps the price between $5-$25
        # The numbers given to vehicles, traffic in the first few columns are given with logic which is useful in this model depending on weightage of those parameters in model pricing formula
        price=pw.apply(
            lambda x: max(5, min(20, x)),
            10 + 5 *(1.2 * (pw.this.occ_avg / pw.this.cap) +
                0.2 * pw.this.queue_length -
                0.2 * pw.this.traffic +
                0.8 * pw.this.special_day +
                0.2 * pw.this.vehicle_type - 1)
        )
    )
)


In [ ]:
from bokeh.plotting import figure
import bokeh.plotting
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window1.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()

In [ ]:
pw.run()